# **UCB方策**によるバンディットのアルゴリズムを考えていきます．

今回は確率バンディットで**UCB方策**を考えていきます，

UCB方策とは簡潔に説明すると, **選択数が少ないアーム**は**真の報酬の期待値**を予測するのが難しいので少ないアームを優先的に引くために考えられたアルゴリズムです．

----

#### まずはUCB方策の式を見ていきましょう．

$$
\bar u_{i}(t) = \hat u_{i}(t) + \sqrt{\frac{logt}{N_{i}(t)}}
$$

**$\hat u_{i}(t)$** は $t$ 回目までに推定されたアームiの報酬の期待値で, **$N_{i}(t)$** は$t$ 回目までにアームiが選ばれた回数です．

##### この式の意味は**期待値が低いアーム**でも選択回数が低いなら, 積極的に引くということです.

---
#### **早速アルゴリズムに入っていきます．**

In [1]:
import numpy as np
#今回はアームが4つの場合を想定します，
class UCB_bandit:
    def __init__(self):
        '''
        p -> それぞれ報酬が発生する確率
        alpha,beta -> ベータ分布の確率
        m -> それぞれの報酬が出た回数
        n -> ぞれぞれが試行された数
        '''
        self.p = [0.1, 0.3, 0.5, 0.8] #最適なアームのインデックスは3
        self.n = np.zeros(len(self.p))


    def choose_action(self,m: int,n: int):
        
        best_index = None
        prev_sample = 0
        for i in range(len(self.p)):
            sample = np.random.beta(self.alpha + m[i], self.beta + n[i] - m[i], 1)
            if best_index == None:
                best_index = i
                prev_sample = sample
                continue

            if sample > prev_sample:
                best_index = i
                prev_sample = sample

        return best_index

    def reward(self,index: int):
        return np.random.binomial(1,self.p[index],1)

    
    def update(self,index,reward):
        self.m[index] += reward
        self.n[index] += 1

    def simulate(self,N):
        for i in range(N):
            arm = self.choose_action(self.m,self.n)
            reward = self.reward(arm)
            self.update(arm,reward)

In [2]:
import numpy as np
from tqdm import tqdm
#今回はアームが4つの場合を想定します，
class UCB_bandit:
    def __init__(self):
        '''
        p -> それぞれ報酬が発生する確率
        alpha,beta -> ベータ分布の確率
        m -> それぞれの報酬が出た回数
        n -> ぞれぞれが試行された数
        '''
        self.p = [0.1, 0.3, 0.5, 0.8] #最適なアームのインデックスは3
        self.N = np.zeros(len(self.p))
        self.rewards = np.zeros(len(self.p))


    def choose_action(self,N: int,t: int):
        #1回も選ばれてないアームをなくす．
        if t in [0,1,2,3]:
            return t

        rewards = np.zeros(len(self.p))
        for i in range(len(self.p)):
            reward = self.rewards[i]/self.N[i] + np.sqrt(np.log(t) / 2 * self.N[i])
            np.append(rewards[i],reward)
        max_index = np.where(rewards == rewards.max())
        index = np.random.choice(max_index[0])


        return index
            

    def reward(self,index: int):

        return np.random.binomial(1,self.p[index],1)

    
    def update(self,index,reward):
        self.rewards[index] += reward
        self.N[index] += 1

    def simulate(self,T):
        for i in tqdm(range(T)):
            arm = self.choose_action(self.N,i)
            reward = self.reward(arm)
            self.update(arm,reward)

In [3]:
agent = UCB_bandit()
agent.simulate(1000)
print(f'0が選ばれた回数:{agent.N[0]}回, 1が選ばれた回数:{agent.N[1]}回, 2が選ばれた回数:{agent.N[2]}回, 3が選ばれた回数:{agent.N[3]}回')

100%|██████████| 1000/1000 [00:00<00:00, 10069.37it/s]

0が選ばれた回数:221.0回, 1が選ばれた回数:253.0回, 2が選ばれた回数:255.0回, 3が選ばれた回数:271.0回
